# Day 3 - Lab 1: AI-Driven Backend Development (Solution)

**Objective:** Generate a complete FastAPI backend application, including Pydantic and SQLAlchemy models, and then perform the critical engineering task of integrating the generated code with the live SQLite database created on Day 2.

**Introduction:**
This solution notebook provides the complete code and prompts for generating and assembling the database-connected API. It highlights the workflow of generating components separately and then integrating them, a common pattern in AI-assisted development.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

**Explanation:**
We load our `schema.sql` artifact, which will be the primary context for our code generation prompts. Having the database schema is essential for the LLM to accurately generate models (both Pydantic and SQLAlchemy) and endpoints that match our data structure.

In [1]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output, recommended_models_table, prompt_enhancer

# Initialize separate LLM clients for different artifacts to use the latest models from different providers.
# - In-memory app generation: use a Scout/Llama family model for instruction-following code generation
# - DB models & session code: use a strong instruction-following model (e.g. gpt-4o)
# - Integration/synthesis tasks: use another high-quality model (e.g. gemini-2.5-pro)
in_memory_client, in_memory_model_name, in_memory_api_provider = setup_llm_client(model_name="meta-llama/Llama-4-Scout-17B-16E-Instruct")
db_client, db_model_name, db_api_provider = setup_llm_client(model_name="claude-opus-4-1-20250805")
integration_client, integration_model_name, integration_api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the SQL schema from Day 2
sql_schema = load_artifact("artifacts/schema.sql")
if not sql_schema:
    print("Warning: Could not load schema.sql. Lab may not function correctly.")

/Users/agaleana/repos/AG-AISOFTDEV/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-21 21:27:20,363 ag_aisoftdev.utils INFO LLM Client configured provider=huggingface model=meta-llama/Llama-4-Scout-17B-16E-Instruct latency_ms=None artifacts_path=None
2025-09-21 21:27:20,598 ag_aisoftdev.utils INFO LLM Client configured provider=anthropic model=claude-opus-4-1-20250805 latency_ms=None artifacts_path=None
2025-09-21 21:27:20,863 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


In [2]:
recommended_models_table(text_generation=True)

| Model | Provider | Text | Vision | Image Gen | Image Edit | Audio Transcription | Context Window | Max Output Tokens |
|---|---|---|---|---|---|---|---|---|
| claude-opus-4-1-20250805 | anthropic | ✅ | ✅ | ❌ | ❌ | ❌ | 200,000 | 100,000 |
| claude-opus-4-20250514 | anthropic | ✅ | ✅ | ❌ | ❌ | ❌ | 200,000 | 100,000 |
| claude-sonnet-4-20250514 | anthropic | ✅ | ✅ | ❌ | ❌ | ❌ | 1,000,000 | 100,000 |
| deepseek-ai/DeepSeek-V3.1 | huggingface | ✅ | ❌ | ❌ | ❌ | ❌ | 128,000 | 100,000 |
| gemini-1.5-flash | google | ✅ | ✅ | ❌ | ❌ | ❌ | 1,000,000 | 8,192 |
| gemini-1.5-pro | google | ✅ | ✅ | ❌ | ❌ | ❌ | 2,000,000 | 8,192 |
| gemini-2.0-flash-exp | google | ✅ | ✅ | ❌ | ❌ | ❌ | 1,048,576 | 8,192 |
| gemini-2.5-flash | google | ✅ | ✅ | ❌ | ❌ | ❌ | 1,048,576 | 65,536 |
| gemini-2.5-flash-lite | google | ✅ | ✅ | ❌ | ❌ | ❌ | 1,048,576 | 65,536 |
| gemini-2.5-pro | google | ✅ | ✅ | ❌ | ❌ | ❌ | 1,048,576 | 65,536 |
| gpt-4.1 | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 1,000,000 | 32,768 |
| gpt-4.1-mini | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 1,000,000 | 32,000 |
| gpt-4.1-nano | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 1,000,000 | 32,000 |
| gpt-4o | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 128,000 | 16,384 |
| gpt-4o-mini | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 128,000 | 16,384 |
| gpt-5-2025-08-07 | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 400,000 | 128,000 |
| gpt-5-mini-2025-08-07 | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 400,000 | 128,000 |
| gpt-5-nano-2025-08-07 | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 400,000 | 128,000 |
| meta-llama/Llama-3.3-70B-Instruct | huggingface | ✅ | ❌ | ❌ | ❌ | ❌ | 8,192 | 4,096 |
| meta-llama/Llama-4-Maverick-17B-128E-Instruct | huggingface | ✅ | ❌ | ❌ | ❌ | ❌ | 1,000,000 | 100,000 |
| meta-llama/Llama-4-Scout-17B-16E-Instruct | huggingface | ✅ | ❌ | ❌ | ❌ | ❌ | 10,000,000 | 100,000 |
| mistralai/Mistral-7B-Instruct-v0.3 | huggingface | ✅ | ❌ | ❌ | ❌ | ❌ | 32,768 | 8,192 |
| o3 | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 200,000 | 100,000 |
| o4-mini | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 200,000 | 100,000 |
| tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.5 | huggingface | ✅ | ❌ | ❌ | ❌ | ❌ | 4,096 | 1,024 |

'| Model | Provider | Text | Vision | Image Gen | Image Edit | Audio Transcription | Context Window | Max Output Tokens |\n|---|---|---|---|---|---|---|---|---|\n| claude-opus-4-1-20250805 | anthropic | ✅ | ✅ | ❌ | ❌ | ❌ | 200,000 | 100,000 |\n| claude-opus-4-20250514 | anthropic | ✅ | ✅ | ❌ | ❌ | ❌ | 200,000 | 100,000 |\n| claude-sonnet-4-20250514 | anthropic | ✅ | ✅ | ❌ | ❌ | ❌ | 1,000,000 | 100,000 |\n| deepseek-ai/DeepSeek-V3.1 | huggingface | ✅ | ❌ | ❌ | ❌ | ❌ | 128,000 | 100,000 |\n| gemini-1.5-flash | google | ✅ | ✅ | ❌ | ❌ | ❌ | 1,000,000 | 8,192 |\n| gemini-1.5-pro | google | ✅ | ✅ | ❌ | ❌ | ❌ | 2,000,000 | 8,192 |\n| gemini-2.0-flash-exp | google | ✅ | ✅ | ❌ | ❌ | ❌ | 1,048,576 | 8,192 |\n| gemini-2.5-flash | google | ✅ | ✅ | ❌ | ❌ | ❌ | 1,048,576 | 65,536 |\n| gemini-2.5-flash-lite | google | ✅ | ✅ | ❌ | ❌ | ❌ | 1,048,576 | 65,536 |\n| gemini-2.5-pro | google | ✅ | ✅ | ❌ | ❌ | ❌ | 1,048,576 | 65,536 |\n| gpt-4.1 | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 1,000,000 | 32,768 |\n| gpt-4.1-

## Step 2: The Challenges - Solutions

### Challenge 1 (Foundational): Generating Code with In-Memory Logic

**Explanation:**
This prompt generates a fully functional but simplified version of our application. By asking for an in-memory database, we allow the LLM to focus on generating the correct API structure, endpoints, and Pydantic models without the added complexity of database integration code. This gives us a clean, working baseline that we can build upon.

In [3]:
in_memory_api_prompt = f"""
You are a senior Python developer creating a FastAPI application for a new hire onboarding tool.

Based on the following SQL schema, generate a single Python script for a `main.py` file that includes:
1.  All necessary FastAPI imports.
2.  Pydantic models for creating and reading `User` resources. Include fields for `id`, `name`, `email`, and `role`.
3.  A simple in-memory list to act as a fake database for users.
4.  Complete FastAPI CRUD endpoints for the `/users` path (POST, GET all, GET by ID).
5.  The endpoints should perform their logic on the in-memory list.

**SQL Schema Context:**
```sql
{sql_schema}
```

Output only the raw Python code.
"""

print("--- Generating FastAPI app with in-memory database ---")
if sql_schema:
    # Enhance the prompt so the model adopts a clear persona and structured output expectations
    enhanced_in_memory_api_prompt = prompt_enhancer(in_memory_api_prompt)
    print("In-memory Enhanced prompt\n", enhanced_in_memory_api_prompt)
    generated_api_code = get_completion(enhanced_in_memory_api_prompt, in_memory_client, in_memory_model_name, in_memory_api_provider)
    cleaned_code = clean_llm_output(generated_api_code, language='python')
    # Save this code to a temporary reference file
    save_artifact(cleaned_code, "app/main_in_memory.py", overwrite=True)
    print("Saved in-memory API to app/main_in_memory.py")
else:
    print("Skipping API generation because schema is missing.")

2025-09-21 21:27:21,001 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


--- Generating FastAPI app with in-memory database ---
In-memory Enhanced prompt
 <prompt>

  <persona>
You are a senior Python backend engineer with deep expertise in FastAPI and Pydantic.  
  </persona>

  <context>
A new-hire onboarding tool will initially manage only user resources.  
SQL schema excerpt (for reference, don’t replicate in output):

```sql
CREATE TABLE users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT NOT NULL UNIQUE,
    role TEXT NOT NULL CHECK (role IN ('New Hire', 'Manager'))
);

CREATE TABLE onboarding_tasks (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    description TEXT NOT NULL,
    due_date DATE NOT NULL,
    status TEXT NOT NULL CHECK (status IN ('Pending', 'Completed')),
    user_id INTEGER NOT NULL,
    FOREIGN KEY (user_id) REFERENCES users(id) ON DELETE CASCADE
);
```

Feature requirements for the initial FastAPI service (file: `main.py`):

1. All necessary FastAPI imports.  
2. Pydantic mo

### Challenge 2 (Intermediate): Generating Database Models and Session Code

**Explanation:**
This prompt is highly specific. It asks for the two key components needed for database connectivity in a modern Python application: the ORM (Object-Relational Mapping) models and the session management code. 
-   **SQLAlchemy Models:** These classes map our Python objects directly to the tables in our database, allowing us to work with Python code instead of raw SQL.
-   **Session Management:** This is the standard FastAPI pattern for handling database connections. The `get_db` function is a dependency that ensures each API request gets a database session and that the session is properly closed afterward.

In [4]:
db_code_prompt = f"""
You are a Python expert specializing in FastAPI and SQLAlchemy.

Based on the provided SQL schema, generate the necessary Python code to connect a FastAPI application to a SQLite database named 'onboarding.db'.

**SQL Schema Context:**
```sql
{sql_schema}
```

Please provide two separate, well-commented code blocks:

1.  **SQLAlchemy Models:** Create the Python classes that map to the `users` and `onboarding_tasks` tables.
2.  **Database Session Management:** Provide the standard boilerplate code for creating the SQLAlchemy engine, the `SessionLocal` class, and the `get_db` dependency for FastAPI.

Only output the raw Python code.
"""

print("--- Generating SQLAlchemy Models and Session Code ---")
if sql_schema:
    # Enhance the DB prompt to ensure precise, well-structured output from the model
    enhanced_db_code_prompt = prompt_enhancer(db_code_prompt)
    print("DB Code Enhanced prompt\n", enhanced_db_code_prompt)
    generated_db_code = get_completion(enhanced_db_code_prompt, db_client, db_model_name, db_api_provider)
    # Clean and save the generated DB code to an artifact so the integration step can use it.
    cleaned_db_code = clean_llm_output(generated_db_code, language='python')
    print("\n--- Generated Database Code (cleaned) ---")
    print(cleaned_db_code)
    save_artifact(cleaned_db_code, "app/db_models.py", overwrite=True)
    print("Saved DB models and session code to app/db_models.py")
else:
    print("Skipping DB code generation because schema is missing.")

2025-09-21 21:27:31,890 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


--- Generating SQLAlchemy Models and Session Code ---
DB Code Enhanced prompt
 <prompt>
  <persona>
    You are an expert Python backend developer with deep, production-level experience in FastAPI and SQLAlchemy. Your answers are concise, precise, and adhere to best practices.
  </persona>

  <context>
    You are connecting a FastAPI application to a SQLite database named “onboarding.db”.  
    The database schema is:

    ```sql
    CREATE TABLE users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        email TEXT NOT NULL UNIQUE,
        role TEXT NOT NULL CHECK (role IN ('New Hire', 'Manager'))
    );

    CREATE TABLE onboarding_tasks (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        description TEXT NOT NULL,
        due_date DATE NOT NULL,
        status TEXT NOT NULL CHECK (status IN ('Pending', 'Completed')),
        user_id INTEGER NOT NULL,
        FOREIGN KEY (user_id) REFERENCES users(id) ON DELETE CASCADE
 

### Challenge 3 (Advanced): Integrating Live Database Logic

**Explanation:**
This final code represents the crucial role of the developer in an AI-assisted workflow. The AI provided the components (Pydantic models, SQLAlchemy models, endpoint structure), but the developer is responsible for the final integration, ensuring all the pieces work together seamlessly. This involves combining the generated code blocks and replacing the in-memory list operations with live SQLAlchemy database calls (`db.add`, `db.query`, `db.commit`, etc.).

In [5]:
# Integration step: combine generated artifacts into a minimal live `app/main.py`
print("--- Integrating generated artifacts into final app/main.py ---")

# Load generated artifacts from the artifacts folder (artifacts/app/*)
in_memory_code = load_artifact("app/main_in_memory.py")
db_models_code = load_artifact("app/db_models.py")

integration_prompt = (
    "You are a pragmatic Python developer. "
    "Produce a minimal `app/main.py` that imports SessionLocal, engine, and User from `app.db_models` "
    "and implements POST /users/ (create), GET /users/ (list), GET /users/{user_id} (retrieve). "
    "Use explicit SQLAlchemy calls (db.add, db.query, db.commit, db.refresh). "
    "Do NOT inline large artifact sources; they are available at artifacts/app/main_in_memory.py and artifacts/app/db_models.py. "
    "Use './artifacts/onboarding.db' as the SQLite file and ensure response models set orm_mode=True. "
    "Return only the raw Python source for app/main.py."
)

# Combine prompt and metadata, then enhance
prompt = prompt_enhancer(integration_prompt)

# Request integration code from the integration model
integration_output = get_completion(
    prompt,
    integration_client,
    integration_model_name,
    integration_api_provider,
    temperature=0.2,
)

# Clean and persist the result
cleaned_integration = clean_llm_output(integration_output, language="python")
print("--- Final integrated app (preview) ---")
print(cleaned_integration)
save_artifact(cleaned_integration, "app/main.py", overwrite=True)
print("Saved integrated app to app/main.py")

# Expose final_api_code for downstream cells that may reference it
final_api_code = cleaned_integration
print("--- Final Integrated API Code for app/main.py ---")
print(final_api_code)
save_artifact(final_api_code, "app/main.py", overwrite=True)

2025-09-21 21:27:58,303 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


--- Integrating generated artifacts into final app/main.py ---
--- Final integrated app (preview) ---
from typing import List
from fastapi import Depends, FastAPI, HTTPException
from sqlalchemy.orm import Session
from pydantic import BaseModel
from app.db_models import SessionLocal, engine, Base, User as DBUser

Base.metadata.create_all(bind=engine)

app = FastAPI()

class UserBase(BaseModel):
    name: str
    email: str

class UserCreate(UserBase):
    pass

class User(UserBase):
    id: int

    class Config:
        orm_mode = True

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

@app.post("/users/", response_model=User)
def create_user(user: UserCreate, db: Session = Depends(get_db)):
    db_user = DBUser(name=user.name, email=user.email)
    db.add(db_user)
    db.commit()
    db.refresh(db_user)
    return db_user

@app.get("/users/", response_model=List[User])
def read_users(skip: int = 0, limit: int = 100, db: Session = Depends(

PosixPath('/Users/agaleana/repos/AG-AISOFTDEV/artifacts/app/main.py')

## Lab Conclusion

Congratulations! You have successfully generated and assembled a complete, database-connected backend API. You used an LLM to generate the boilerplate for both the API endpoints and the database models, and then performed the crucial engineering task of integrating them. You now have a working `main.py` file in your `app` directory that can create, read, update, and delete data in a live database. In the next lab, we will write a comprehensive test suite for this API.

> **Key Takeaway:** AI excels at generating boilerplate code (like models and endpoint structures), but the developer's critical role is in the final integration and wiring of these components into a coherent, working system.